In [1]:
import pandas as pd 
import geopandas as gpd
import urllib
import shapely
pd.options.mode.chained_assignment = None  # default='warn'
from math import *
from geopy.geocoders import Nominatim


In [2]:
data_g = gpd.read_file('data/prix-carburants-fichier-quotidien-test-ods.geojson')

In [3]:
def data_carb_g(df, carb):
    data_1 = df[['adresse', 'ville', 'prix_valeur', 'prix_nom', 'geometry']]
    d = data_1[data_1.prix_nom == carb]
    d.drop_duplicates(keep = 'first', inplace=True) 
    #df.drop_duplicates(subset ="'column'", keep = 'first', inplace=True) si on veut supprimer selon une colonne
    return d

In [4]:
data_2_g = data_carb_g(data_g, 'SP95').sort_values('prix_valeur')

In [5]:
def Adresse_to_gps(Adresse) : 

    geolocator = Nominatim(user_agent="Projet Python")
    location = geolocator.geocode(Adresse)
    return location.latitude, location.longitude


Adresse_to_gps("1 rue d'Arsonval, Orsay, France")

(48.7087952, 2.1742033)

In [53]:
point = shapely.geometry.Point(Adresse_to_gps("1 rue d'Arsonval, Orsay, France"))
col_adresse = gpd.GeoSeries((point for i in range(len(data_2_g))),index= data_2_g.index, crs = 4326)
col_adresse.to_crs(2154)

2402     POINT (6956939.356 3081851.193)
60125    POINT (6956939.356 3081851.193)
36655    POINT (6956939.356 3081851.193)
22911    POINT (6956939.356 3081851.193)
25102    POINT (6956939.356 3081851.193)
                      ...               
44978    POINT (6956939.356 3081851.193)
11148    POINT (6956939.356 3081851.193)
43359    POINT (6956939.356 3081851.193)
4049     POINT (6956939.356 3081851.193)
58312    POINT (6956939.356 3081851.193)
Length: 3709, dtype: geometry

In [8]:
type(data_2_g.distance(col_adresse))

C:\Users\hugol\AppData\Local\Temp\ipykernel_17068\3041818366.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  type(data_2_g.distance(col_adresse))


pandas.core.series.Series

In [50]:
def dix_plus_proche_g(df, posi, carb): 
    data = data_carb_g(df, carb)
    p = shapely.geometry.Point(Adresse_to_gps(posi))
    col_ad = gpd.GeoSeries((p for i in range(len(data))),index= data.index, crs = 4326)
    new_data = data.to_crs(2154)
    new_col = col_ad.to_crs(2154)
    data_1 = new_data.distance(new_col).sort_values().head(10)
    
    return data.loc[data_1.index.to_list()]

In [52]:
dix_plus_proche_g(data_2_g, "1 rue d'Arsonval, Orsay, France", "SP95")

,adresse,ville,prix_valeur,prix_nom,geometry
37807,Cala Longa,Bonifacio,1.731,SP95,POINT (9.16600 41.39200)
7382,sennola,BONIFACIO,1.730,SP95,POINT (9.16500 41.39100)
2592,358 San Cipriano,Lecci,1.790,SP95,POINT (9.33600 41.63900)
17548,Route de Bonifacio,Porto-Vecchio,1.770,SP95,POINT (9.27300 41.57100)
14738,Route de bonifacio BP115,Porto-Vecchio,1.790,SP95,POINT (9.27469 41.57844)
29264,Les 4 Chemins BP115,Porto-Vecchio,1.790,SP95,POINT (9.27600 41.59800)
4933,Station TOTAL - Quartier La Poretta,PORTO-VECCHIO,1.750,SP95,POINT (9.27600 41.60200)
7396,RTE N 198,Porto-Vecchio,1.790,SP95,POINT (9.27700 41.60400)
32440,route de bastia,Porto-Vecchio,1.790,SP95,POINT (9.28000 41.61400)
14737,Station TOTAL La Trinité - RN 198 - Pitrera,PORTO-VECCHIO,1.640,SP95,POINT (9.29420 41.63490)
